In [1]:
import os 
import sys

import sklearn as sk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

from scipy.stats import kurtosis, entropy
from scipy.fft import fft

In [2]:
sys.path.append(r"/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool") #這行為絕對路徑，如需使用，必須要修改為當前決路徑
import csi_tool
import denoise
import data_loader
import csidataset


In [3]:
base_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/1223_phone/5G/20MHz/csv/all"



# 粗定位
spacing = 0.6  # 每隔 0.6m

reference_points = {}

for ref_id, coord in data_loader.COORDINATES.items():
    folder_path = os.path.join(base_path, f"reference_point_{ref_id}.xlsx")
    reference_points[folder_path] = (ref_id, coord)

data, rp_labels, coord_labels = data_loader.load_data(reference_points)

In [5]:
#細定位

# reference_points = {
#     f"{base_path}/reference_point{i}.xlsx": i for i in range(1, 37)
# }

# print(reference_points)

# def load_data(reference_points):
#     data = []          
#     rp_labels = []     

#     for path, ref_id in reference_points.items():
#         df = pd.read_excel(path)
#         data.append(df.values)
#         rp_labels.extend([ref_id] * len(df))  # 只保留 Reference Point ID

#     data = pd.DataFrame(np.vstack(data))
#     rp_labels = pd.Series(rp_labels, name="Reference Point ID")  # 轉為 Pandas Series

#     return data, rp_labels

In [17]:
#data, rp_labels = load_data(reference_points)

In [4]:
data

,0,1,2,3,4,5,6,7,8,9,...,88,89,90,91,92,93,94,95,96,97
0,919.592301,929.595611,877.760787,898.203763,851.400023,893.573164,888.473973,871.023536,836.316328,841.770753,...,-1.387518,-1.581549,-1.708760,-2.080869,-2.287379,-2.467120,-2.666997,-2.982241,-50.0,136.0
1,698.204841,732.963846,684.079674,694.257877,651.079872,672.521375,687.674342,780.946221,803.560203,863.103702,...,-2.677159,-2.877730,-3.051501,2.738820,2.503722,2.256683,2.035376,1.737005,-49.0,148.0
2,745.268408,768.188128,706.606680,713.252410,681.482208,713.950278,736.619983,852.877482,846.442556,880.032954,...,0.176051,-0.063704,-0.251636,-0.776139,-1.023850,-1.275624,-1.535994,-1.899522,-50.0,224.0
3,830.278267,829.860832,804.005597,796.492310,786.787138,800.870776,791.982323,778.923616,763.872372,766.167084,...,-1.031820,-1.194379,-1.338067,-1.666445,-1.878363,-2.100073,-2.369892,-2.666982,-51.0,136.0
4,906.099884,903.507056,910.843565,896.688352,892.235955,875.416472,878.300632,854.687077,857.886939,843.664033,...,-0.445534,-0.593750,-0.783166,-1.090942,-1.286103,-1.497279,-1.784787,-2.076610,-50.0,136.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24495,397.615392,416.889674,395.588170,413.019370,382.733589,381.136459,393.814677,434.337426,465.842248,483.735465,...,2.183710,2.005190,1.826107,1.364322,1.150287,0.872978,0.573063,0.255977,-56.0,148.0
24496,643.708785,646.260010,627.624888,621.498994,605.152873,628.458431,617.272225,592.763865,581.986254,587.911558,...,-0.494557,-0.782703,-1.053078,-1.631902,-1.969954,-2.328378,-2.758960,3.087458,-59.0,136.0
24497,762.685387,745.553486,706.884007,695.708272,698.951357,710.169698,720.184004,675.370269,654.120020,669.209982,...,0.621717,0.498336,0.356470,-0.131807,-0.426306,-0.748795,-1.031706,-1.339351,-58.0,136.0
24498,401.220638,416.688133,388.561449,390.436935,370.411933,382.498366,394.452786,436.664631,446.643034,495.258518,...,-1.801749,-2.120022,-2.410138,3.138034,2.751942,2.306627,1.868269,1.432071,-56.0,148.0


In [5]:
amp_data = np.array(data.iloc[:, :48])
phase_data = np.array(data.iloc[:, 48:-2]) 

In [6]:
amp_d = denoise.preprocess_csi_for_fingerprint2(amp_data)

In [7]:
amp_d.shape

(24500, 48)

In [8]:
encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = encoder.fit_transform(np.array(rp_labels).reshape(-1, 1))

# KNN K值測試

In [27]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# 設定 N-Fold 交叉驗證
N_FOLD = 10
skf = StratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=42)

# 轉換 One-Hot 標籤為標籤索引
y_labels = np.argmax(one_hot_labels, axis=1)

# 測試的 K 值範圍
k_values = range(3, 20, 2)  # 1, 3, 5, ..., 19
best_k = None
best_acc = 0
all_k_accuracies = {}  # 紀錄不同 K 值的準確率

# 測試不同的 K 值
for k in k_values:
    accuracies = []  # 儲存 N-Fold 交叉驗證的準確率
    
    # 執行 N-Fold 交叉驗證
    for train_index, test_index in skf.split(amp_d, y_labels):
        X_train, X_test = amp_d[train_index], amp_d[test_index]
        y_train, y_test = y_labels[train_index], y_labels[test_index]

        # 訓練 KNN 模型
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)

        # 進行預測
        y_pred = knn.predict(X_test)

        # 計算準確率
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)

    # 計算 K 值的平均準確率
    mean_acc = np.mean(accuracies)
    all_k_accuracies[k] = mean_acc  # 記錄 K 值的結果

    print(f"K={k} 平均準確率: {mean_acc * 100:.2f}%")

    # 更新最佳 K 值
    if mean_acc > best_acc:
        best_acc = mean_acc
        best_k = k

# 輸出最佳 K 值
print("\n📊 最佳 KNN 模型")
print(f"🎯 最佳 K 值: {best_k}")
print(f"🏆 最佳準確率: {best_acc * 100:.2f}%")


K=3 平均準確率: 99.38%
K=5 平均準確率: 99.31%
K=7 平均準確率: 99.25%
K=9 平均準確率: 99.16%
K=11 平均準確率: 99.13%
K=13 平均準確率: 99.03%
K=15 平均準確率: 98.94%
K=17 平均準確率: 98.83%
K=19 平均準確率: 98.73%

📊 最佳 KNN 模型
🎯 最佳 K 值: 3
🏆 最佳準確率: 99.38%


# WKNN K值測試

In [14]:
# 設定 N-Fold 交叉驗證
N_FOLD = 10
skf = StratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=42)

# 轉換 One-Hot 標籤為標籤索引
y_labels = np.argmax(one_hot_labels, axis=1)

# 測試的 K 值範圍
k_values = range(3, 20, 2)  # 1, 3, 5, ..., 19
best_k = None
best_acc = 0
all_k_accuracies = {}  # 紀錄不同 K 值的準確率

# 測試不同的 K 值
for k in k_values:
    accuracies = []  # 儲存 N-Fold 交叉驗證的準確率
    
    # 執行 N-Fold 交叉驗證
    for train_index, test_index in skf.split(amp_d, y_labels):
        X_train, X_test = amp_data[train_index], amp_data[test_index]
        y_train, y_test = y_labels[train_index], y_labels[test_index]

        # 訓練 KNN 模型
        wknn = KNeighborsClassifier(n_neighbors=3, weights='distance')
        wknn.fit(X_train, y_train)

        # 進行預測
        y_pred = wknn.predict(X_test)

        # 計算準確率
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)

    # 計算 K 值的平均準確率
    mean_acc = np.mean(accuracies)
    all_k_accuracies[k] = mean_acc  # 記錄 K 值的結果

    print(f"K={k} 平均準確率: {mean_acc * 100:.2f}%")

    # 更新最佳 K 值
    if mean_acc > best_acc:
        best_acc = mean_acc
        best_k = k

# 輸出最佳 K 值
print("\n📊 最佳 KNN 模型")
print(f"🎯 最佳 K 值: {best_k}")
print(f"🏆 最佳準確率: {best_acc * 100:.2f}%")

K=3 平均準確率: 98.89%
K=5 平均準確率: 98.89%
K=7 平均準確率: 98.89%
K=9 平均準確率: 98.89%
K=11 平均準確率: 98.89%
K=13 平均準確率: 98.89%
K=15 平均準確率: 98.89%
K=17 平均準確率: 98.89%
K=19 平均準確率: 98.89%

📊 最佳 KNN 模型
🎯 最佳 K 值: 3
🏆 最佳準確率: 98.89%


# WKNN KNN 測試測試

In [54]:
amp_train, amp_temp, y_train, y_temp = train_test_split(amp_data, one_hot_labels, test_size=1/3, random_state=42)
amp_val, amp_test, y_val, y_test = train_test_split(amp_temp, y_temp, test_size=1/3, random_state=42)

In [91]:
#KNN
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn.fit(amp_train, y_train)
y_pred = knn.predict(amp_test)
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")



KNN Accuracy: 98.86%


In [92]:
def compute_mean_distance_error(y_true, y_pred, coordinates):
    """
    y_true, y_pred: 一維的 NumPy 陣列，分別存放真實和預測的 label（整數）
    coordinates: dict, label -> (x, y)
    """
    errors = []
    for true_label, pred_label in zip(y_true, y_pred):
        # 取出對應的座標
        if true_label not in coordinates or pred_label not in coordinates:
            # 若某個 label 不在座標字典內，就跳過（或視需求處理）
            continue
        true_coord = np.array(coordinates[true_label])
        pred_coord = np.array(coordinates[pred_label])
        # 計算歐氏距離
        error = np.linalg.norm(pred_coord - true_coord)
        errors.append(error)
    return np.mean(errors) if errors else None

mean_error = compute_mean_distance_error(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), data_loader.COORDINATES)
print("Mean distance error:", mean_error)

Mean distance error: 0.03401416792523786


In [100]:
#WKNN
wknn = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='euclidean')
wknn.fit(amp_train, y_train)
y_pred = wknn.predict(amp_test)
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

KNN Accuracy: 98.93%


In [96]:
mean_error = compute_mean_distance_error(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), data_loader.COORDINATES)
print("Mean distance error:", mean_error)

Mean distance error: 0.030442671497361982


# N-ford 測試推理速度

In [97]:
print(type(amp_d[0]))
print(type(amp_test[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [101]:
import time
import numpy as np
knn.fit(amp_train, y_train)
start_time = time.perf_counter()
y_pred = knn.predict(amp_test)  # 進行 KNN 預測
end_time = time.perf_counter()

total_time = end_time - start_time  # 計算 KNN 預測總時間
print(f"總推論時間: {total_time:.6f} 秒")
avg_time_per_sample = total_time / len(amp_test)  # 計算每筆資料的平均推論時間
# 顯示結果
print(f"📊 N-FOLD 測試結果（共 {N_FOLD} 次）")
print(f"⏳ 平均每次總推論時間: {total_time:.6f} 秒")
print(f"⏱️ 平均每筆資料推論時間: {avg_time_per_sample * 1000:.6f} 毫秒")




總推論時間: 3.604277 秒
📊 N-FOLD 測試結果（共 1 次）
⏳ 平均每次總推論時間: 3.604277 秒
⏱️ 平均每筆資料推論時間: 1.323642 毫秒


In [102]:
import time
import numpy as np
wknn.fit(amp_train, y_train)
start_time = time.perf_counter()
y_pred = wknn.predict(amp_test)  # 進行 KNN 預測
end_time = time.perf_counter()

total_time = end_time - start_time  # 計算 KNN 預測總時間
print(f"總推論時間: {total_time:.6f} 秒")
avg_time_per_sample = total_time / len(amp_test)  # 計算每筆資料的平均推論時間
# 顯示結果
print(f"📊 N-FOLD 測試結果（共 {N_FOLD} 次）")
print(f"⏳ 平均每次總推論時間: {total_time:.6f} 秒")
print(f"⏱️ 平均每筆資料推論時間: {avg_time_per_sample * 1000:.6f} 毫秒")

總推論時間: 0.046864 秒
📊 N-FOLD 測試結果（共 1 次）
⏳ 平均每次總推論時間: 0.046864 秒
⏱️ 平均每筆資料推論時間: 0.017210 毫秒
